In [1]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.optimizers import SGD

import tensorflow as tf
tf.compat.v1.disable_v2_behavior()
tf.compat.v1.enable_eager_execution(
    config=None, device_policy=None, execution_mode=None
)
import keras

Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
mnist = tf.keras.datasets.mnist

(train_data, train_labels), (test_data, test_labels) = mnist.load_data()
train_data, test_data = train_data / 255.0, test_data / 255.0

# Add a channels dimension
train_data = train_data[..., tf.newaxis].astype("float32")
test_data = test_data[..., tf.newaxis].astype("float32")


VALIDATION_SIZE = 5000
validation_data = train_data[:VALIDATION_SIZE, :, :, :]
validation_labels = train_labels[:VALIDATION_SIZE]
train_data = train_data[VALIDATION_SIZE:, :, :, :]
train_labels = train_labels[VALIDATION_SIZE:]
params = [32, 32, 64, 64, 200, 200]
batch_size = 128

# Prepare the training dataset.
train_dataset = tf.data.Dataset.from_tensor_slices((train_data, train_labels))
train_dataset = train_dataset.shuffle(buffer_size=1024).batch(batch_size)

# Prepare the validation dataset.
val_dataset = tf.data.Dataset.from_tensor_slices((validation_data, validation_labels))
val_dataset = val_dataset.batch(batch_size)

class MNIST:
    def __init__(self, train_data, train_labels, validation_data, validation_labels, test_data, test_labels):
        self.train_data = train_data
        self.train_labels = train_labels
        self.test_data = test_data
        self.test_labels = test_labels
        self.validation_data = validation_data
        self.validation_labels = validation_labels

mnist_data = MNIST(train_data, train_labels, validation_data, validation_labels, test_data, test_labels)


def show(img):
    """
    Show MNSIT digits in the console.
    """
    remap = "  .*#"+"#"*100
    img = (img.flatten()+.5)*3
    if len(img) != 784: return
    print("START")
    for i in range(28):
        print("".join([remap[int(round(x))] for x in img[i*28:i*28+28]]))

def get_model(data, file_name, params, num_epochs=50, batch_size=128, train_temp=1, init=None):
    """
    Standard neural network training procedure.
    """
    model = Sequential()

    print(data.train_data.shape)
    
    model.add(Conv2D(params[0], (3, 3),
                            input_shape=data.train_data.shape[1:]))
    model.add(Activation('relu'))
    model.add(Conv2D(params[1], (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(params[2], (3, 3)))
    model.add(Activation('relu'))
    model.add(Conv2D(params[3], (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Flatten())
    model.add(Dense(params[4]))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(params[5]))
    model.add(Activation('relu'))
    model.add(Dense(10))
    model.add(Activation('softmax'))

    return model

def get_madry_model(data, file_name, params, num_epochs=50, batch_size=128, train_temp=1, init=None):
    """
    Standard neural network training procedure.
    """
    model = Sequential()

    print(data.train_data.shape)
    
    model.add(Conv2D(32, (5, 5), padding="same",
                            input_shape=data.train_data.shape[1:]))
    # model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), padding="same"))
    model.add(Conv2D(64, (5, 5), padding="same"))
    # model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), padding="same"))

    model.add(Flatten())
    model.add(Dense(1024))
    # model.add(Activation('relu'))
    model.add(Dense(10))
    model.add(Activation('softmax'))

    return model

def get_madry_little_diff_model(data, file_name, params, num_epochs=50, batch_size=128, train_temp=1, init=None):
    """
    Standard neural network training procedure.
    """
    model = Sequential()

    print(data.train_data.shape)
    
    model.add(Conv2D(32, (3, 3),
                            input_shape=data.train_data.shape[1:]))
    # model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(64, (3, 3)))
    # model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Flatten())
    model.add(Dense(1024))
    # model.add(Activation('relu'))
    model.add(Dense(10))
    model.add(Activation('softmax'))

    return model

def get_madry_with_relu_model(data, file_name, params, num_epochs=50, batch_size=128, train_temp=1, init=None):
    """
    Standard neural network training procedure.
    """
    model = Sequential()

    print(data.train_data.shape)
    
    model.add(Conv2D(32, (5, 5), padding="same",
                            input_shape=data.train_data.shape[1:]))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), padding="same"))
    model.add(Conv2D(64, (5, 5), padding="same"))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), padding="same"))

    model.add(Flatten())
    model.add(Dense(1024))
    model.add(Activation('relu'))
    model.add(Dense(10))
    model.add(Activation('softmax'))

    return model

11493376/11490434 [==============================] - 0s 0us/step


In [3]:
pgd_model = get_model(mnist_data, "mnistModel", params)

import keras
# Instantiate an optimizer to train the model.
optimizer = keras.optimizers.SGD(learning_rate=0.01, decay=1e-6, momentum=0.9)
# Instantiate a loss function.
loss_fn1 = keras.losses.SparseCategoricalCrossentropy(from_logits=True)
loss_fn2 = keras.losses.BinaryCrossentropy()
loss_fn3 = keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction=tf.keras.losses.Reduction.SUM)

# Prepare the metrics.
train_acc_metric = keras.metrics.SparseCategoricalAccuracy()
val_acc_metric = keras.metrics.SparseCategoricalAccuracy()


# @tf.function
def train_step(x1, x2, y):
    with tf.GradientTape() as tape:
        # logits1 = pgd_model(x1, training=True)
        logits2 = pgd_model(x2, training=True)
        # loss_value = tf.add(loss_fn1(y, logits1), loss_fn1(y, logits2))
        loss_value = loss_fn1(y,logits2)
    grads = tape.gradient(loss_value, pgd_model.trainable_weights)
    optimizer.apply_gradients(zip(grads, pgd_model.trainable_weights))
    train_acc_metric.update_state(y, logits2)
    return loss_value

def normal_train_step(x1,y):
    with tf.GradientTape() as tape:
        logits1 = pgd_model(x1, training=True)
        # logits2 = pgd_model(x2, training=True)
        # loss_value = tf.add(loss_fn1(y, logits1), loss_fn1(y, logits2))
        loss_value = loss_fn1(y,logits1)
    grads = tape.gradient(loss_value, pgd_model.trainable_weights)
    optimizer.apply_gradients(zip(grads, pgd_model.trainable_weights))
    train_acc_metric.update_state(y, logits1)
    return loss_value

# @tf.function
def test_step(x, y):
    val_logits = pgd_model(x, training=False)
    val_acc_metric.update_state(y, val_logits)

(55000, 28, 28, 1)


In [14]:
"""
Implementation of attack methods. Running this file as a program will
apply the attack to the model specified by the config file and store
the examples in an .npy file.
"""
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

# import tensorflow as tf
# import numpy as np


class L2PGDAttack:
  def __init__(self, model, epsilon, k, a, random_start, loss_func):
    """Attack parameter initialization. The attack performs k steps of
       size a, while always staying within epsilon from the initial
       point."""
    self.model = pgd_model
    self.epsilon = epsilon
    self.k = k
    self.a = a
    self.rand = random_start
    self.E_steps = 30
    self.sigma = 1000.0/255

  def perturb(self, x_nat, y):
    """Given a set of examples (x_nat, y), returns a set of adversarial
       examples within epsilon of x_nat in l_infinity norm."""
    if self.rand:
        x = x_nat + np.random.uniform(-self.epsilon, self.epsilon, x_nat.shape)
        x = np.clip(x, 0, 1) # ensure valid pixel range
    else:
        x = np.copy(x_nat)
    # print(x_nat.shape)

    for i in range(self.k):
        x_tensor = tf.convert_to_tensor(x)
        # x_tensor = tf.reshape(x_tensor,)
        with tf.GradientTape() as t:
            t.watch(x_tensor)
            logits1 = pgd_model(x_tensor, training=True)
            loss_value = loss_fn3(y, logits1)
        result = loss_value
        grad = t.gradient(loss_value, x_tensor)
        grad = np.zeros(grad.shape)

        for j in range(self.E_steps):
            d = np.zeros((28*28))
            for k in range(28*28):
                d[k] = np.random.normal(scale=self.sigma)
            d = d.reshape(28,28,1)
            x_new = x_tensor + d
            with tf.GradientTape() as t:
                t.watch(x_new)
                logits1 = pgd_model(x_new, training=True)
                loss_value = loss_fn3(y, logits1)
            result = loss_value
            grad1 = t.gradient(loss_value, x_new)
            grad = grad + grad1

        grad = grad/(self.E_steps)
        grad_norm = np.linalg.norm(np.linalg.norm(grad,axis=(1,2)),axis=1).reshape(-1,1,1,1)
        grad = grad/grad_norm

        x += self.a*grad
        delta = tf.reshape((x_nat - x),[-1,28*28])
        delta = tf.clip_by_norm(delta, self.epsilon,axes=1)
        delta = tf.reshape(delta,[-1,28,28,1])
        x = x_nat + delta

        x = np.clip(x, 0, 1) # ensure valid pixel range

    return x

In [15]:
import json
import sys
import math

attack = L2PGDAttack(pgd_model,.3,20,.01,True,"xent") 

In [16]:
# Training model
import time
import math 
std = 0.01
epochs = 5
pgd_model = get_model(mnist_data, "mnistModel", params)

total_steps = sum(1 for _ in train_dataset)
for epoch in range(epochs):
    start_time = time.time()

    # Iterate over the batches of the dataset.
    for step, (x_batch_train, y_batch_train) in enumerate(train_dataset):
        # x_batch_adv = attack.perturb(x_batch_train, y_batch_train).reshape(-1,28,28,1)
        # Compute loss
        loss_value = normal_train_step(x_batch_train, y_batch_train)
        print('\r', "Epoch %d" % (epoch,), 'Training step:', step+1, f'/{total_steps}', 'Loss:', float(loss_value), 'Acc:', float(train_acc_metric.result()), end='')
    
    # Display metrics at the end of each epoch.
    train_acc = train_acc_metric.result()
    print("\nTraining acc over epoch: %.4f" % (float(train_acc),), end=' ')

    # Reset training metrics at the end of each epoch
    train_acc_metric.reset_states()

    # Run a validation loop at the end of each epoch.
    for x_batch_val, y_batch_val in val_dataset:
        test_step(x_batch_val, y_batch_val)
    val_acc = val_acc_metric.result()
    val_acc_metric.reset_states()
    print("Validation acc: %.4f" % (float(val_acc),), end=' ')
    print("Time taken: %.2fs\n" % (time.time() - start_time))

(55000, 28, 28, 1)
 Epoch 0 Training step: 430 /430 Loss: 0.1699027270078659 Acc: 0.818363606929779
Training acc over epoch: 0.8184 Validation acc: 0.9740 Time taken: 8.03s

 Epoch 1 Training step: 430 /430 Loss: 0.011361004784703255 Acc: 0.962218165397644
Training acc over epoch: 0.9622 Validation acc: 0.9802 Time taken: 8.04s

 Epoch 2 Training step: 430 /430 Loss: 0.03585405275225639 Acc: 0.9723636507987976
Training acc over epoch: 0.9724 Validation acc: 0.9808 Time taken: 7.95s

 Epoch 3 Training step: 430 /430 Loss: 0.06758473068475723 Acc: 0.9777091145515442
Training acc over epoch: 0.9777 Validation acc: 0.9872 Time taken: 8.03s

 Epoch 4 Training step: 430 /430 Loss: 0.07364551723003387 Acc: 0.9815090894699097
Training acc over epoch: 0.9815 Validation acc: 0.9876 Time taken: 7.96s



In [ ]:
import json
import sys
import math
attack = L2PGDAttack(pgd_model, 0.3,20,0.01,True,"xent")

# Iterate over the samples batch-by-batch
num_eval_examples = 10000
eval_batch_size = 128
num_batches = int(math.ceil(num_eval_examples / eval_batch_size))

x_adv = [] # adv accumulator

print('Iterating over {} batches'.format(num_batches))

for ibatch in range(num_batches):
    bstart = ibatch * eval_batch_size
    bend = min(bstart + eval_batch_size, num_eval_examples)
    print(ibatch)
    # print('batch size: {}'.format(bend - bstart))

    x_batch = test_data[bstart:bend, :, :, :]
    y_batch = test_labels[bstart:bend]

    x_batch_adv = attack.perturb(x_batch, y_batch).reshape(-1,28,28,1)
    x_adv.append(x_batch_adv)

print('Storing examples')
path = 'pgd_attack2.npy'
x_adv = np.concatenate(x_adv, axis=0)
np.save(path, x_adv)
print('Examples stored in {}'.format(path))

Iterating over 79 batches
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26


In [ ]:
pgd_adv = np.load('pgd_attack2.npy')

In [ ]:
a = tf.zeros((2,2))
print(a.dtype)

<dtype: 'float32'>


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cp pgd_attack2.npy /content/drive/MyDrive/

In [ ]:
pgd_adv = np.load('/content/drive/Shareddrives/AML-Project/L2SecondOrder/pgd_attack2.npy')

In [ ]:
i = 10
x_linf = pgd_adv[i,:,:,:]
# show(x_linf)

import matplotlib.pyplot as plt

im_linf = x_linf.reshape(28, 28)
plt.gray()
plt.imshow(im_linf)
plt.figure()
plt.imshow(test_data[i,:,:,:].reshape(28,28))
print(test_labels[i])
print(np.sum((x_linf-test_data[i,:,:,:])**2))

In [ ]:
i = 3
x_linf = pgd_adv[i,:,:,:]
# show(x_linf)

import matplotlib.pyplot as plt

im_linf = x_linf.reshape(28, 28)
plt.gray()
plt.imshow(im_linf)
plt.figure()
plt.imshow(test_data[i,:,:,:].reshape(28,28))
print(test_labels[i])
print(np.sum((x_linf-test_data[i,:,:,:])**2))

In [12]:
def test_attack_single_input(adv, x, true_label, num_trials=10, std=0.01):
    # original input
    y = pgd_model(x)
    pred_label = np.argmax(y)
    count = np.zeros(10)
    for i in range(num_trials):
        x_noisy = x + np.random.normal(scale=std, size=x.shape)
        y_noisy = pgd_model(x_noisy)
        noisy_label = np.argmax(y_noisy)
        count[noisy_label] += 1
    noisy_label = np.argmax(count)

    # adv. input
    y_adv = pgd_model(adv)
    adv_label = np.argmax(y_adv)
    count = np.zeros(10)
    for i in range(num_trials):
        adv_noisy = adv + np.random.normal(scale=std, size=adv.shape)
        y_noisy_adv = pgd_model(adv_noisy)
        noisy_label_adv = np.argmax(y_noisy_adv)
        count[noisy_label_adv] += 1
    adv_noisy_label = np.argmax(count)
    print('\r', 'true:', true_label, 'pred:', pred_label, 'noisy_pred:', noisy_label, 'adv_pred:', adv_label, 'adv_noisy_pred:', adv_noisy_label, end='')

    return true_label, pred_label, noisy_label, adv_label, adv_noisy_label

def test_attack_multiple_inputs(num_runs=100):
    corr = 0
    noisy_corr = 0
    adv_corr = 0
    adv_noisy_corr = 0
    for run in range(num_runs):
        i = np.random.randint(0, 10000)
        x = test_data[i,:,:,:][np.newaxis,:,:,:]
        adv = pgd_adv[i,:,:,:][np.newaxis,:,:,:]
        true, pred, noisy_pred, adv_pred, adv_noisy_pred = test_attack_single_input(adv, x, test_labels[i], num_trials=10, std=0.1)
        corr += (pred == true)
        noisy_corr += (true == noisy_pred)
        adv_corr += (true == adv_pred)
        adv_noisy_corr += (true == adv_noisy_pred)
    print(f'\nAcc Original: {corr/num_runs}, Acc Adv: {adv_corr/num_runs}, Acc Adv Noisy: {adv_noisy_corr/num_runs}, Acc Ori Noisy: {noisy_corr/num_runs}')

In [ ]:
pgd_model.save_weights('/content/drive/MyDrive/task2a-v2')

In [ ]:
pgd_model.load_weights('/content/drive/MyDrive/task2a')

In [ ]:
# Training normal model
import time
pgd_model = get_madry_with_relu_model(mnist_data, "mnistModel", params)
epochs = 10
total_steps = sum(1 for _ in train_dataset)
for epoch in range(epochs):
    start_time = time.time()

    # Iterate over the batches of the dataset.
    for step, (x_batch_train, y_batch_train) in enumerate(train_dataset):
        # x_batch_adv = attack.perturb(x_batch_train, y_batch_train).reshape(-1,28,28,1)
        # Compute loss
        loss_value = normal_train_step(x_batch_train, y_batch_train)
        print('\r', "Epoch %d" % (epoch,), 'Training step:', step+1, f'/{total_steps}', 'Loss:', float(loss_value), 'Acc:', float(train_acc_metric.result()), end='')
    
    # Display metrics at the end of each epoch.
    train_acc = train_acc_metric.result()
    print("\nTraining acc over epoch: %.4f" % (float(train_acc),), end=' ')

    # Reset training metrics at the end of each epoch
    train_acc_metric.reset_states()

    # Run a validation loop at the end of each epoch.
    for x_batch_val, y_batch_val in val_dataset:
        test_step(x_batch_val, y_batch_val)
    val_acc = val_acc_metric.result()
    val_acc_metric.reset_states()
    print("Validation acc: %.4f" % (float(val_acc),), end=' ')
    print("Time taken: %.2fs\n" % (time.time() - start_time))

(55000, 28, 28, 1)
 Epoch 0 Training step: 430 /430 Loss: 0.17815004289150238 Acc: 0.9060949087142944
Training acc over epoch: 0.9061 Validation acc: 0.9756 Time taken: 6.03s

 Epoch 1 Training step: 430 /430 Loss: 0.02269153483211994 Acc: 0.9777272939682007
Training acc over epoch: 0.9777 Validation acc: 0.9764 Time taken: 5.89s

 Epoch 2 Training step: 430 /430 Loss: 0.007426992058753967 Acc: 0.9846181869506836
Training acc over epoch: 0.9846 Validation acc: 0.9864 Time taken: 5.60s

 Epoch 3 Training step: 430 /430 Loss: 0.01214504148811102 Acc: 0.9882363677024841
Training acc over epoch: 0.9882 Validation acc: 0.9862 Time taken: 5.74s

 Epoch 4 Training step: 430 /430 Loss: 0.005040264688432217 Acc: 0.9907090663909912
Training acc over epoch: 0.9907 Validation acc: 0.9896 Time taken: 5.62s

 Epoch 5 Training step: 150 /430 Loss: 0.06689710170030594 Acc: 0.9921875

KeyboardInterrupt: ignored

In [13]:
test_attack_multiple_inputs()

 true: 2 pred: 2 noisy_pred: 2 adv_pred: 2 adv_noisy_pred: 2
Acc Original: 1.0, Acc Adv: 1.0, Acc Adv Noisy: 1.0, Acc Ori Noisy: 1.0
